A ideia desse projeto é usar o SKORCH para realizar validação cruzada

# Carregando a base de dados

In [1]:
import pandas as pd

X = pd.read_csv("../../bases/bases/entradas_breast.csv")
y = pd.read_csv("../../bases/bases/saidas_breast.csv")

In [2]:
import numpy as np

X = np.array(X, dtype="float32")
y = np.array(y, dtype="float32").squeeze(1)

In [3]:
X

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e+02, 2.750e+02,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e+02, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e+02, 4.087e-01,
        1.240e+02],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]], shape=(569, 30), dtype=float32)

# Classe para a rede neural

In [4]:
import torch.nn as nn
class classificador_torch(nn.Module):
    def __init__(self):
        super().__init__() # pegando todos os atributos da super classe (nn.Module)
        
        self.dense0 = nn.Linear(in_features=30, out_features=16)
        nn.init.uniform_(self.dense0.weight) # inicializando a primeira camada com pesos uniformemente aleatorio
        self.activation0 = nn.ReLU()
        self.dropout0 = nn.Dropout(0.2)
        self.dense1 = nn.Linear(in_features=16, out_features=16)
        nn.init.uniform_(self.dense1.weight)
        self.activation1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.dense2 = nn.Linear(in_features=16, out_features=1)
        nn.init.uniform_(self.dense2.weight)
        self.output = nn.Sigmoid()
    def forward(self, x):
        x = self.dense0(x)
        x = self.activation0(x)
        x = self.dropout0(x)
        x = self.dense1(x)
        x = self.activation1(x)
        x = self.dropout1(x)
        x = self.dense2(x)
        x = self.output(x)
        return x

# Usando o SKORCH

In [5]:
from skorch import NeuralNetBinaryClassifier
import torch 

classificador_sk = NeuralNetBinaryClassifier(
    module=classificador_torch,
    criterion=nn.BCELoss,
    optimizer=torch.optim.Adam,
    lr=0.001,
    optimizer__weight_decay=0.0001,
    max_epochs=100,
    batch_size=10,
    train_split=False
)

# Realizando a validação cruzada

In [6]:
from sklearn.model_selection import cross_val_score

results = cross_val_score(estimator=classificador_sk, X=X, y=y, cv=10, scoring="accuracy")

  epoch    train_loss     dur
-------  ------------  ------
      1       37.1094  0.0689
      2       37.1094  0.0597
      3       37.1094  0.0570
      4       37.1094  0.0596
      5       37.1094  0.0595
      6       37.1094  0.0575
      7       37.1094  0.0571
      8       37.1094  0.0570
      9       37.1094  0.0576
     10       37.1094  0.0591
     11       37.1094  0.0570
     12       37.1094  0.0560
     13       37.1094  0.0563
     14       37.1094  0.0590
     15       37.1094  0.0582
     16       37.1094  0.0588
     17       37.1094  0.0588
     18       37.1094  0.0686
     19       37.1094  0.0579
     20       37.1094  0.0732
     21       37.1094  0.0576
     22       37.1094  0.0584
     23       37.1094  0.0570
     24       37.1094  0.0582
     25       37.1094  0.0609
     26       15.9942  0.0600
     27        0.5831  0.0606
     28        0.5583  0.0622
     29        0.5391  0.0649
     30        0.5194  0.0608
     31        0.5145  0.0615
     32   

In [7]:
results

array([0.84210526, 0.8245614 , 0.84210526, 0.92982456, 0.84210526,
       0.89473684, 0.85964912, 0.94736842, 0.9122807 , 0.89285714])

In [36]:
results

array([0.85964912, 0.80701754, 0.61403509, 0.63157895, 0.85964912,
       0.94736842, 0.85964912, 0.92982456, 0.63157895, 0.875     ])

In [8]:
results.mean()

np.float64(0.8787593984962406)

In [37]:
results.mean()

np.float64(0.8015350877192983)